#Training Data
This notebook will show how the data was structured from different files of MIMIC to design the model. First Section will show you , How it was arranged to check it from local host location. Outputs are Visible to some operations.



In [0]:
import json
import pandas as pd
import os

file_path = 'C:\\Users\\KAKA-HPC\\Downloads\\'
train_csv = os.path.join(os.path.dirname(file_path), "notes_labeled_train.ndjson")

with open(train_csv) as f:
    content=[]
    for line in f:
        content = json.loads(line)
print(content)
# pre processed file

In [0]:
%pwd


'C:\\Users\\KAKA-HPC\\Downloads'

Just for a better visualization.

In [0]:
len(content)

5

Patient ID: From records with all diagnosis

In [0]:
print(content[0])

83532


Hospital Admission ID:

In [0]:
print(content[1])

180932


Diagnosis Summary

In [0]:
print(content[2][1])


['february', 'discharge', 'date']


Length of Text tokens in Content[2]


In [0]:
print(content[3])

2592


ICD_9 Codes labeled based on discharge Summaries.

In [0]:
print(content[4])

['428.23', '570', '425.4', '416.8', '578.1', '428.0', '403.90', '585.9', '780.2', '573.8', 'V53.32', '493.90', '285.9', '790.92', '530.81', '311', 'V58.61', '37.23', '38.97', '89.49']


Single View of Description of text relevant to one Pateint

In [0]:
from functools import reduce
import operator
te=reduce(operator.add, content[2])
te= ' '.join(te)
te
#view= ' '.join(content[2])

"admission date february discharge date february date of birth january sex m service medicine allergies azithromycin gadopentetate dimeglumine morphine sulfate keflex iodine iodine containing attending lastnametoken chief complaint syncope major surgical or invasive procedure cardiac catheterization central line placement and removal picc line placement history of present illness mr. lastnametoken is a year old man with dilated cardiomyopathy ef on coumadin s p icd in htn ckd baseline cr and asthma who presents with syncope the patient was recently discharged on february for acute on chronic chf after receiving aggressive diuresis he then continued to develop abdominal discomfort nausea and vomitting and he was unable to keep his medications down he then developed worsening coughing spells and it was in the setting of a severe coughing spell when he had a witnesssed syncopal event for about seconds per the patient 's mother the patient was pale and cyanotic he quickly recovered and was

Data Restructured for BertEmbeddings. Note that Bert Embeddings are contexutal embeddings and requires sentence level understanding of works. Tokens like [CLS] & [SEP] are defined to tokenize in a predefined way.




In [0]:
from google.colab import drive
drive.mount('/content/drive')


In [0]:
import json
import pandas as pd

file_path = '/content/drive/My Drive/clinical_notes_ICD9/input_files/notes_labeled_train.ndjson'
with open(file_path) as f:
   with open('/content/new_labeled.ndjson', 'a+') as json_file:
    content=[]
    for line in f:
        content = json.loads(line)
        content[2]=["[CLS]"]+ content[2]
        for a,_lst in enumerate(content[2]):
          if a==0:
            continue
          content[2][a]=content[2][a]+["[SEP]"]        
        json.dump(content,json_file)
        json_file.write('\n')
      

An excerpt for newly formated notes.

['[CLS]', ['admission', 'date', '[SEP]'], ['october', '[SEP]'], ['discharge', 'date', '[SEP]'], ['october', '[SEP]'], ['date', 'of', 'birth', '[SEP]'], ['april', '[SEP]'], ['sex', 'm', 'service', 'medicine', '[SEP]'], ['chief', 'complaint', '[SEP]'], ['admitted', 'from', 'rehabilitation', 'for', 'hypotension', 'systolic', 'blood', 'pressure', 'to', 'the', '70s', 'and', 'decreased', 'urine', 'output', '[SEP]'],




In [0]:
from itertools import chain
y = list(chain(*content[2]))
y= ' '.join(y)
y

This how text is feed to tokenizer in chunks of 512 at once.

"[ C L S ] admission date [SEP] october [SEP] discharge date [SEP] october [SEP] date of birth [SEP] april [SEP] sex m service medicine [SEP] chief complaint [SEP] admitted from rehabilitation for hypotension systolic blood pressure to the 70s and decreased urine output [SEP] history of present illness [SEP] the patient is a year old male who had been hospitalized at the hospitaltoken from october through october of [SEP] after undergoing a left femoral at bypass graft and was subsequently discharged to a rehabilitation facility [SEP] on october [SEP] he presented again to the hospitaltoken after being found to have a systolic blood pressure in the 70s and no urine output for hours [SEP] a foley catheter placed at the rehabilitation facility yielded cc of murky brown urine [SEP] there may also have been purulent discharge at the penile meatus at this time [SEP]
